<a href="https://colab.research.google.com/github/AliGreo/LLMs/blob/main/Electra_finetuned_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from pprint import pprint
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch

In [49]:
tok = ElectraTokenizer.from_pretrained("google/electra-base-discriminator",
                                    clean_up_tokenization_spaces=False)

model = AutoModel.from_pretrained("google/electra-base-discriminator")
tok('The fake small fox jumbs over me')

{'input_ids': [101, 1996, 8275, 2235, 4419, 18414, 29232, 2058, 2033, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [50]:
pprint(tok.tokenize('the financial time said the the iranians want to destroy israil completely'))

['the',
 'financial',
 'time',
 'said',
 'the',
 'the',
 'iranian',
 '##s',
 'want',
 'to',
 'destroy',
 'is',
 '##rail',
 'completely']


In [51]:
pprint(tok('The fake small fox jumbs over me', return_tensors='pt'))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101,  1996,  8275,  2235,  4419, 18414, 29232,  2058,  2033,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [52]:
pprint(model(**tok('The fake small fox jumbs over me', return_tensors='pt')))

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.1234,  0.2610,  0.0831,  ..., -0.0355, -0.0184,  0.4677],
         [ 0.1094, -0.0128,  0.5871,  ..., -0.3561,  0.2020,  0.0520],
         [ 0.1332, -0.1341,  0.0294,  ..., -0.1504,  0.4073, -0.2793],
         ...,
         [-0.2143, -0.0445,  0.1943,  ..., -0.0915, -0.3528, -0.2898],
         [ 0.2510,  0.2064,  0.4437,  ...,  0.3659, -0.0483,  0.7171],
         [-0.1234,  0.2610,  0.0831,  ..., -0.0356, -0.0184,  0.4677]]],
       grad_fn=<NativeLayerNormBackward0>),
                                          past_key_values=None,
                                          hidden_states=None,
                                          attentions=None,
                                          cross_attentions=None)


In [53]:
tok = ElectraTokenizer.from_pretrained("google/electra-base-generator",
                                    clean_up_tokenization_spaces=False)
classifier = ElectraForSequenceClassification.from_pretrained("google/electra-base-generator",
                                                                num_labels=2)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
ids = tok("we went to the movie to watch RMD but the film was disappointed to us.",
          return_tensors="pt")
pprint(ids)
pprint("--"*50)
pprint(classifier(**ids).logits)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101,  2057,  2253,  2000,  1996,  3185,  2000,  3422, 28549,  2094,
          2021,  1996,  2143,  2001,  9364,  2000,  2149,  1012,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
'----------------------------------------------------------------------------------------------------'
tensor([[-0.0019,  0.0531]], grad_fn=<AddmmBackward0>)


In [55]:
import torch

logits = classifier(**ids).logits
print('Model Logits:\n',logits.detach().cpu(), "\n")
probas = torch.softmax(logits, dim=-1)
print('Model Probabilaties:\n', probas.detach().cpu(), "\n")

model_label_output = probas.argmax()
print('Model Label Output:\n', model_label_output.detach().cpu().item())

Model Logits:
 tensor([[-0.0019,  0.0531]]) 

Model Probabilaties:
 tensor([[0.4862, 0.5138]]) 

Model Label Output:
 1


In [9]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00


In [60]:
import datasets
dataset = datasets.load_dataset("imdb", split="train[:5000]")
pprint(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})


In [61]:
def tokenize(row):
    output = tok(row['text'],padding=True, truncation=True, return_tensors="pt").to("cuda")
    return {'attention_mask':output['attention_mask'].squeeze(0),
            'token_type_ids':output['token_type_ids'].squeeze(0),
            'input_ids':output['input_ids'].squeeze(0)}

pprint(tokenize(dataset[0]))

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1

In [62]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [65]:
# split to train and test
data = tokenized_dataset\
                         .shuffle(seed=42)\
                         .select(range(5000))\
                         .train_test_split(test_size=0.2)\
                         .remove_columns("text")
pprint(data)

DatasetDict({
    train: Dataset({
        features: ['label', 'attention_mask', 'token_type_ids', 'input_ids'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['label', 'attention_mask', 'token_type_ids', 'input_ids'],
        num_rows: 1000
    })
})


In [18]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [66]:
import evaluate
import numpy as np

acc = evaluate.load("accuracy")

def accuracy(model_output):
    logits, labels = model_output
    predictions = np.argmax(logits, axis=-1)

    return acc.compute(predictions=predictions, references=labels)

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [69]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="electra-base-finetuned-imdb",
                         #num_train_epochs=1,
                         per_device_train_batch_size=20,
                         per_device_eval_batch_size=20,
                         learning_rate=2e-4,
                         eval_strategy='steps',
                         max_steps=200,
                         save_safetensors=False,
                         save_strategy='steps',
                         logging_steps=50,
                         push_to_hub=True)

model = Trainer(model=classifier.to("cuda"),
                tokenizer=tok,
                train_dataset=data['train'],
                eval_dataset=data['test'],
                args=args,
                compute_metrics=accuracy)

model.train()

max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000100,0.000051,1.000000


TrainOutput(global_step=200, training_loss=0.0005222674389369785, metrics={'train_runtime': 77.1124, 'train_samples_per_second': 51.872, 'train_steps_per_second': 2.594, 'total_flos': 119707557888000.0, 'train_loss': 0.0005222674389369785, 'epoch': 1.0})

In [70]:
model.evaluate()

{'eval_loss': 5.13571358169429e-05,
 'eval_accuracy': 1.0,
 'eval_runtime': 9.2728,
 'eval_samples_per_second': 107.842,
 'eval_steps_per_second': 5.392,
 'epoch': 1.0}

In [79]:
classifier.config.id2label = {0:'Negative', 1:'Positive'}

In [90]:
input_ids = tok("the movie was so bad i sad", return_tensors="pt").to("cuda")
model_output = model.model(**input_ids).logits.detach().cpu().argmax().item()
print(model_output)
print(f'The model predicted that the sentiment is: #{classifier.config.id2label[model_output]}')

0
The model predicted that the sentiment is: #Negative
